In [1]:
from queries import run_custom_query
from database import add_new_columns, update_stance_categories, update_stance_categories_threshold
import database
import queries

import sys
import json
import jsonlines
import ijson
import pandas as pd

full = "../data/dataset_full_all_fields_v2.jsonl"
public = "../data/dataset_zenodo.jsonl"

In [ ]:
# Set file paths
jsonl_file = "../data/dataset_full_all_fields_v2.jsonl"
db_file = "data.db"

# Remove the existing database if necessary
import os
if os.path.exists(db_file):
    os.remove(db_file)

# Recreate the database
conn = database.create_connection(db_file)
database.create_table(conn)

# Load JSONL data into the database
print("Loading data into the database...")
database.load_data(conn, jsonl_file)
print("Data loaded successfully.")

# Close the connection
conn.close()

In [2]:
conn = database.create_connection("data.db")

In [ ]:
# Add New Columns
add_new_columns(conn)

In [ ]:
# Update Stance Categories
update_stance_categories_threshold(conn, 0.8)

In [ ]:
df_full = queries.fetch_first_n_rows(conn, n=5)
df_full

In [28]:
# Define the query
query= """
WITH stance_data AS (
    SELECT 
        channel AS Media_Outlet, 
        COUNT(*) AS Tweet_Count,
        SUM(pro_russia) AS Pro_Russia,
        SUM(pro_ukraine) AS Pro_Ukraine,
        SUM(unsure) AS Unsure
    FROM records
    GROUP BY channel
    ORDER BY Tweet_Count DESC
)

SELECT * FROM stance_data;
"""

result = run_custom_query(conn, query)

In [ ]:
columns = ["Media Outlet", "Tweet_Count", "Pro_Russia", "Pro_Ukraine", "Unsure"]
df = pd.DataFrame(result, columns=columns)
df

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd

# Function to calculate proportions with a minimum pro-Russia + pro-Ukraine tweet count threshold
def calculate_proportions(df, min_combined_count=100):
    df_filtered = df[(df['Pro_Russia'] + df['Pro_Ukraine']) >= min_combined_count]
    df_filtered["Proportion_Pro_Russia"] = (df_filtered["Pro_Russia"] / df_filtered["Tweet_Count"]).round(2)
    df_filtered["Proportion_Pro_Ukraine"] = (df_filtered["Pro_Ukraine"] / df_filtered["Tweet_Count"]).round(2)
    df_filtered["Pro_Ukraine_to_Pro_Russia"] = (df_filtered["Proportion_Pro_Ukraine"] / df_filtered["Proportion_Pro_Russia"]).round(2)
    df_cleaned = df_filtered.dropna(subset=["Pro_Ukraine_to_Pro_Russia"])
    return df_cleaned[df_cleaned["Pro_Ukraine_to_Pro_Russia"].notna()][["Media Outlet", "Proportion_Pro_Russia", "Proportion_Pro_Ukraine", "Pro_Ukraine_to_Pro_Russia"]]

# Function to visualize proportions as a bar plot
def visualize_proportions(df, save_path="../plots/plot_5.png"):
    df = df.dropna(subset=["Pro_Ukraine_to_Pro_Russia"])
    df.sort_values(by='Pro_Ukraine_to_Pro_Russia', ascending=False, inplace=True)
    fig, ax = plt.subplots(figsize=(15, 8))

    # Create bar plot
    ax.bar(df['Media Outlet'], df['Pro_Ukraine_to_Pro_Russia'], color='skyblue')
    ax.set_xlabel('Media Outlet', fontsize=14)
    ax.set_ylabel('Pro-Ukraine to Pro-Russia Ratio', fontsize=14)
    ax.set_title('Top 20: Pro-Ukraine to Pro-Russia Stance by Media Outlet', fontsize=16)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    plt.show()


result_df = calculate_proportions(df)
# Sort result_df by descending Pro_Ukraine_to_Pro_Russia value
sorted_df = result_df.sort_values(by='Pro_Ukraine_to_Pro_Russia', ascending=False)
top_20 = sorted_df.head(20)
bottom_20 = sorted_df.tail(20)
visualize_proportions(top_20)
top_20